# Лабораторная работа 2 (Linear Model)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet
from utils import regression_cross_validate, display_metrics_table, classification_cross_validate, display_metrics_classification_table

import warnings
warnings.filterwarnings("ignore")

### Regression

#### 1. Обработка данных

In [2]:
df = pd.read_csv('data/laptop_prices.csv')
df.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,No,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,Yes,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,Yes,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650


In [3]:
features, target = df.drop(columns=['Price_euros']), df['Price_euros']

num_cols = features.select_dtypes(include=[np.number]).columns  # Численные признаки
cat_cols = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=-1)

train_features[cat_cols] = le.fit_transform(train_features[cat_cols])
test_features[cat_cols] = le.transform(test_features[cat_cols])

most_frequent_imputer = SimpleImputer(strategy='most_frequent') 
train_features = pd.DataFrame(most_frequent_imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(most_frequent_imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна 

Для оценки модели будем использовать метод кросс валидации, который позволяет более качественно оценить полученные метрики.

In [4]:
metrics = regression_cross_validate(LinearRegression, train_features.to_numpy(), train_target.to_numpy(), n_folds=5)
display_metrics_table(*metrics)

linear_model = LinearRegression()
linear_model.fit(train_features, train_target)

y_pred = linear_model.predict(test_features)

# Метрики
mse = mean_squared_error(test_target, y_pred)
mae = mean_absolute_error(test_target, y_pred)
r2 = r2_score(test_target, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    272.669    |    16.5331    |
| MSE      | 143590        | 12631.2       |
| R2       |      0.703708 |     0.0461453 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 130868.65
Средняя абсолютная ошибка (MAE): 264.34
Коэффициент детерминации (R^2): 0.73


Значение метрики $R^2$ составляет около 0.73, что указывает на то, что модель объясняет примерно 73% вариации данных. Кроме того, среднее абсолютное значение ошибки (MAE) равно 264.34, что характеризует точность предсказаний модели в абсолютных единицах (в среднем предсказания ошибаются примерно на 264 евро).

#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Добавить регуляризацию

In [5]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(train_features[cat_cols])
encoded_test_data = onehot.transform(test_features[cat_cols])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(cat_cols))
X_train_upd = train_features.drop(columns=cat_cols).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(cat_cols))
X_test_upd = test_features.drop(columns=cat_cols).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)

scaler = StandardScaler()
X_train_upd[num_cols] = scaler.fit_transform(train_features[num_cols])
X_test_upd[num_cols] = scaler.transform(test_features[num_cols])

In [6]:
metrics = regression_cross_validate(ElasticNet, X_train_upd.to_numpy(), train_target.to_numpy(), n_folds=5, alpha=0.01, l1_ratio=0.7)
display_metrics_table(*metrics)

linear_model = ElasticNet(alpha=0.01, l1_ratio=0.7)
linear_model.fit(X_train_upd, train_target)

y_pred = linear_model.predict(X_test_upd)

# Метрики
mse = mean_squared_error(test_target, y_pred)
mae = mean_absolute_error(test_target, y_pred)
r2 = r2_score(test_target, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |         Mean |      Std Dev |
|:---------|-------------:|-------------:|
| MAE      |   191.999    |   12.728     |
| MSE      | 80770.6      | 8977.75      |
| R2       |     0.833241 |    0.0294235 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 65975.97
Средняя абсолютная ошибка (MAE): 185.85
Коэффициент детерминации (R^2): 0.86


Можно заметить, что в среднем показатели метрик значительно улучшились по сравнению с предыдущими результатами. Анализ результатов на тестовой выборке также демонстрирует существенные улучшения, что подтверждает эффективность проведённых модификаций.

#### 4. Реализация своего класса

In [7]:
class MyElasticNet:
    def __init__(self, alpha=1.0, l1_ratio=0.5, max_iter=1000, tol=1e-4):
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.max_iter = max_iter
        self.tol = tol
        self.coef_ = None
        self.intercept_ = None

    def fit(self, features, target):
        # Преобразование данных в numpy-формат
        features = np.array(features, dtype=np.float64)
        target = np.array(target, dtype=np.float64)

        # Добавляем bias (сместить данные)
        features = np.c_[np.ones(features.shape[0]), features]

        # Параметры регуляризации
        lambda_1 = self.alpha * self.l1_ratio  # L1-регуляризация
        lambda_2 = self.alpha * (1 - self.l1_ratio)  # L2-регуляризация

        # Инициализация коэффициентов
        n_features = features.shape[1]
        self.coef_ = np.zeros(n_features)

        # Итеративное решение с L1-регуляризацией
        for iteration in range(self.max_iter):
            # Предыдущие коэффициенты (для остановки)
            coef_old = self.coef_.copy()

            # L2-регуляризация: добавляем к (features.T @ features) регуляризующий член
            XtX = features.T @ features + lambda_2 * np.eye(n_features)
            Xty = features.T @ target

            # Решение нормального уравнения
            self.coef_ = np.linalg.solve(XtX, Xty - lambda_1 * np.sign(self.coef_))

            # Остановка, если изменения малы
            if np.linalg.norm(self.coef_ - coef_old, ord=2) < self.tol:
                break

        # Сохранение смещения отдельно
        self.intercept_ = self.coef_[0]
        self.coef_ = self.coef_[1:]

    def predict(self, features):
        # Преобразование данных в numpy-формат
        features = np.array(features, dtype=np.float64)

        # Добавляем bias (единицы)
        features = np.c_[np.ones(features.shape[0]), features]

        # Предсказание
        return features @ np.r_[self.intercept_, self.coef_]


In [8]:
metrics = regression_cross_validate(MyElasticNet, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, alpha=0, l1_ratio=0)
display_metrics_table(*metrics)

linear_model = MyElasticNet(alpha=0, l1_ratio=0)
linear_model.fit(train_features, train_target)

y_pred = linear_model.predict(test_features)

# Метрики
mse = mean_squared_error(test_target, y_pred)
mae = mean_absolute_error(test_target, y_pred)
r2 = r2_score(test_target, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    272.669    |    16.5331    |
| MSE      | 143590        | 12631.2       |
| R2       |      0.703708 |     0.0461453 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 130868.65
Средняя абсолютная ошибка (MAE): 264.34
Коэффициент детерминации (R^2): 0.73


In [9]:
metrics = regression_cross_validate(MyElasticNet, X_train_upd.to_numpy(), train_target.to_numpy(), n_folds=5, alpha=0.01, l1_ratio=0.7)
display_metrics_table(*metrics)

linear_model = MyElasticNet(alpha=0.01, l1_ratio=0.7)
linear_model.fit(X_train_upd, train_target)

y_pred = linear_model.predict(X_test_upd)

# Метрики
mse = mean_squared_error(test_target, y_pred)
mae = mean_absolute_error(test_target, y_pred)
r2 = r2_score(test_target, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    225.295    |    15.2225    |
| MSE      | 113048        | 10085.6       |
| R2       |      0.767206 |     0.0346196 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 85822.38
Средняя абсолютная ошибка (MAE): 199.01
Коэффициент детерминации (R^2): 0.82


Результаты демонстрируют, что разработанная собственная реализация модели в целом обеспечивает уровень качества, сопоставимый с результатами, достигаемыми стандартной моделью из библиотеки `sklearn`.

### Classification

#### 1. Обработка данных

In [10]:
df = pd.read_csv('data/AIDS_Classification.csv')
df.head()

,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,infected
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [11]:
features, target = df.drop(columns=['infected']), df['infected']

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

num_features = features.select_dtypes(include=[np.number]).columns  # Численные признаки
cat_cols = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

train_features[cat_cols] = le.fit_transform(train_features[cat_cols])
test_features[cat_cols] = le.transform(test_features[cat_cols])

most_frequent_imputer = SimpleImputer(strategy='most_frequent') 
train_features = pd.DataFrame(most_frequent_imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(most_frequent_imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна 

In [12]:
metrics = classification_cross_validate(LogisticRegression, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, max_iter=1000)
display_metrics_classification_table(*metrics)

linear_model = LogisticRegression(max_iter=1000)  # Выбираем количество соседей
linear_model.fit(train_features, train_target)

y_pred = linear_model.predict(test_features)

accuracy = accuracy_score(test_target, y_pred)
precision = precision_score(test_target, y_pred, average='weighted')
recall = recall_score(test_target, y_pred, average='weighted')
f1 = f1_score(test_target, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.858487 | 0.0241342 |
| Precision | 0.852902 | 0.0252986 |
| Recall    | 0.858487 | 0.0241342 |
| F1-score  | 0.85162  | 0.025912  |

=== Результаты на Тесте ===
1. Accuracy: 85.23%
2. Precision: 84.58%
3. Recall: 85.23%
4. F1-score: 84.33%


#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Добавить регуляризацию

In [13]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(train_features[cat_cols])
encoded_test_data = onehot.transform(test_features[cat_cols])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(cat_cols))
X_train_upd = train_features.drop(columns=cat_cols).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(cat_cols))
X_test_upd = test_features.drop(columns=cat_cols).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)


scaler = StandardScaler()
X_train_upd[num_features] = scaler.fit_transform(train_features[num_features])
X_test_upd[num_features] = scaler.transform(test_features[num_features])

In [14]:
metrics = classification_cross_validate(LogisticRegression, X_train_upd.to_numpy(), train_target.to_numpy(), n_folds=5, C=6, max_iter=1000)
display_metrics_classification_table(*metrics)

linear_model = LogisticRegression(C=6, max_iter=1000)  
linear_model.fit(X_train_upd, train_target)

y_pred = linear_model.predict(X_test_upd)

accuracy = accuracy_score(test_target, y_pred)
precision = precision_score(test_target, y_pred, average='weighted')
recall = recall_score(test_target, y_pred, average='weighted')
f1 = f1_score(test_target, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.869706 | 0.0211347 |
| Precision | 0.865467 | 0.0220093 |
| Recall    | 0.869706 | 0.0211347 |
| F1-score  | 0.864696 | 0.021546  |

=== Результаты на Тесте ===
1. Accuracy: 86.17%
2. Precision: 85.68%
3. Recall: 86.17%
4. F1-score: 85.29%


Можно увидеть что в среднем значения метрик улучшились. 
Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [15]:
class MyLogisticRegression:
    def __init__(self, C=0, learning_rate=0.01, n_iterations=1000):
        self.C = C
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, features, target):
        n_samples, n_features = features.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            linear_model = np.dot(features, self.weights) + self.bias
            predictions = self.sigmoid(linear_model)
            if self.C != 0:
                dw = (1 / n_samples) * np.dot(features.T, (predictions - target)) + (1 / (self.C * n_samples)) * self.weights
            else:
                dw = (1 / n_samples) * np.dot(features.T, (predictions - target))
            db = (1 / n_samples) * np.sum(predictions - target)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, features):
        linear_model = np.dot(features, self.weights) + self.bias
        predictions = self.sigmoid(linear_model)
        return np.where(predictions >= 0.5, 1, 0)

In [16]:
metrics = classification_cross_validate(MyLogisticRegression, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_iterations=1000)
display_metrics_classification_table(*metrics)

linear_model = MyLogisticRegression(n_iterations=1000)
linear_model.fit(train_features, train_target)

y_pred = linear_model.predict(test_features)

accuracy = accuracy_score(test_target, y_pred)
precision = precision_score(test_target, y_pred, average='weighted')
recall = recall_score(test_target, y_pred, average='weighted')
f1 = f1_score(test_target, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.745676 | 0.0888301 |
| Precision | 0.796077 | 0.039282  |
| Recall    | 0.745676 | 0.0888301 |
| F1-score  | 0.74322  | 0.0791728 |

=== Результаты на Тесте ===
1. Accuracy: 81.68%
2. Precision: 83.42%
3. Recall: 81.68%
4. F1-score: 82.28%


In [17]:
metrics = classification_cross_validate(MyLogisticRegression, X_train_upd.to_numpy(), train_target.to_numpy(), n_folds=5, C=6, n_iterations=1000)
display_metrics_classification_table(*metrics)

linear_model = MyLogisticRegression(C=6, n_iterations=1000)  
linear_model.fit(X_train_upd, train_target)

y_pred = linear_model.predict(X_test_upd)

accuracy = accuracy_score(test_target, y_pred)
precision = precision_score(test_target, y_pred, average='weighted')
recall = recall_score(test_target, y_pred, average='weighted')
f1 = f1_score(test_target, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.853495 | 0.0175925 |
| Precision | 0.847363 | 0.0176393 |
| Recall    | 0.853495 | 0.0175925 |
| F1-score  | 0.847003 | 0.017704  |

=== Результаты на Тесте ===
1. Accuracy: 85.05%
2. Precision: 84.35%
3. Recall: 85.05%
4. F1-score: 84.22%


Результаты показывают, что собственная реализация модели в среднем такой же по качеству что и модели из `sklearn`, однако демонстрирует приросты при использовании улучшенного бейзлайна.

### Заключение

Внесённые изменения, включая нормализацию данных и добавление регуляризации в алгоритм, улучшают точность модели. Проведённые эксперименты демонстрируют, что как собственная реализация, так и применение моделей из sklearn, дают схожие результаты.

| Модель                    |      MSE  |        MAE |      $R^2$ |
|:--------------------------|----------:|-----------:|-----------:|
| Sklearn (до улучшения)    | 130868.65  | 264.34   |  0.73      |
| Sklearn (после улучшения) | 65975.97  | 185.85   |  0.86      |
| Собственная имплементация (до улучшения)   | 130868.65   | 264.34    |  0.73      |
| Собственная имплементация (после улучшения)| 85822.38  | 199.01    |  0.82      |

| Модель                    |  Accuracy |  Precision |     Recall |    F1-score |
|:--------------------------|----------:|-----------:|-----------:|-----------:|
| Sklearn (до улучшения)    |   85.23%  |   84.58%   |  85.23%    |  84.33%    |
| Sklearn (после улучшения) |   86.17%  |   85.68%   |  86.17%    |  85.29%    |
| Собственная имплементация (до улучшения)   |   81.68%  |   83.42%   |  81.68%    |  82.28%    |
| Собственная имплементация (после улучшения)|   85.05%  |   84.35%   |  85.05%    |  84.22%    |